In [1]:
# Importa las bibliotecas necesarias
from pyspark.sql import SparkSession

# Crea una instancia de SparkSession
spark = SparkSession.builder.appName("Ejemplo").getOrCreate()

# Reemplaza 'ruta/al/archivo.txt' con la ruta completa de tu archivo
path_1 = 'generifs_basic\generifs_basic.txt'
#https://www.ncbi.nlm.nih.gov/gene/about-generif
path_2 = 'hiv_interactions\hiv_interactions.txt'
path_3 = 'hiv_siRNA_interactions\hiv_siRNA_interactions.txt'
path_4 = 'interactions\interactions.txt'


# Lee el archivo de texto en un DataFrame
df_gen_ba = spark.read.csv(path_1, header=True, inferSchema=True, sep='\t')
df_hiv_int = spark.read.csv(path_2, header=True, inferSchema=True, sep='\t')
df_hiv_sirna = spark.read.csv(path_3, header=True, inferSchema=True, sep='\t')
df_int = spark.read.csv(path_4, header=True, inferSchema=True, sep='\t')

# Muestra los primeros registros del DataFrame
#df_gen_ba.show(vertical=True, truncate=False)
#df_hiv_int.show(vertical=True, truncate=False)
#df_hiv_sirna.show(vertical=True, truncate=False)
#df_int.show(vertical=True, truncate=False)

# Mostrar el campo de los comentarios
#df_hiv_int.select("GeneRIF text").show(truncate=False)
#df_hiv_sirna.select("GeneRIF text").show(truncate=False)


In [2]:
from pyspark.sql.functions import col, desc, count, countDistinct, explode, split, size

In [3]:
path = 'biogrid\BIOGRID-ALL-4.4.229.tab3.txt.txt'
# Lee el archivo de texto en un DataFrame
df_all = spark.read.csv(path, header=True, inferSchema=True, sep='\t')

In [4]:
df_homo_sapiens = df_all.where( (col('Organism Name Interactor A') == 'Homo sapiens') & (col('Organism Name Interactor B') == 'Homo sapiens') )

In [5]:
df_homo_sapiens.count()

1119345

In [6]:
df_int.count()

4140525

In [7]:
df_homo_sapiens.show(2, vertical = True)

-RECORD 0--------------------------------------------------
 #BioGRID Interaction ID            | 103                  
 Entrez Gene Interactor A           | 6416                 
 Entrez Gene Interactor B           | 2318                 
 BioGRID ID Interactor A            | 112315               
 BioGRID ID Interactor B            | 108607               
 Systematic Name Interactor A       | -                    
 Systematic Name Interactor B       | -                    
 Official Symbol Interactor A       | MAP2K4               
 Official Symbol Interactor B       | FLNC                 
 Synonyms Interactor A              | JNKK|JNKK1|MAPKK4... 
 Synonyms Interactor B              | ABP-280|ABP280A|A... 
 Experimental System                | Two-hybrid           
 Experimental System Type           | physical             
 Author                             | Marti A (1997)       
 Publication Source                 | PUBMED:9006895       
 Organism ID Interactor A           | 96

In [8]:
df_int.show(2, vertical = True)

-RECORD 0-----------------------------------
 #tax_id             | 358                  
 gene_id             | 1224321              
 accn.vers2          | NP_059802.1          
 name3               | hypothetical prot... 
 keyphrase           | -                    
 tax_id              | -                    
 interactant_id      | -                    
 interactant_id_type | -                    
 accn.vers8          | AE008690.1           
 name9               | Agrobacterium tum... 
 complex_id          | -                    
 complex_id_type     | -                    
 complex_name        | -                    
 pubmed_id_list      | 15155952             
 last_mod            | 2005-01-18 14:45:00  
 generif_text        | VirB4 interacts w... 
 interaction_id      | 134324               
 interaction_id_type | BIND                 
-RECORD 1-----------------------------------
 #tax_id             | 358                  
 gene_id             | 1224322              
 accn.vers

In [9]:
resultado = df_int.join(df_homo_sapiens, (df_int["gene_id"] == df_homo_sapiens["Entrez Gene Interactor A"])\
                        & (df_int["interactant_id"] == df_homo_sapiens["Entrez Gene Interactor B"]), "inner")

In [10]:
resultado.count()

1308840

In [11]:
resultado.groupBy('interaction_id_type').agg(count('*').alias('conteo')).sort(col('conteo')).show()

+-------------------+-------+
|interaction_id_type| conteo|
+-------------------+-------+
|               BIND|  24423|
|               HPRD|  93400|
|            BioGRID|1191017|
+-------------------+-------+



In [12]:
resultado.groupBy('Experimental System Type').agg(count('*').alias('conteo')).sort(col('conteo')).show()

+------------------------+-------+
|Experimental System Type| conteo|
+------------------------+-------+
|                 genetic|  25292|
|                physical|1283548|
+------------------------+-------+



In [13]:
resultado.where(col('Experimental System Type') == 'genetic').show(2, vertical = True)

-RECORD 0--------------------------------------------------
 #tax_id                            | 9606                 
 gene_id                            | 15                   
 accn.vers2                         | -                    
 name3                              | -                    
 keyphrase                          | -                    
 tax_id                             | 9606                 
 interactant_id                     | 6755                 
 interactant_id_type                | GeneID               
 accn.vers8                         | -                    
 name9                              | -                    
 complex_id                         | -                    
 complex_id_type                    | -                    
 complex_name                       | -                    
 pubmed_id_list                     | 28319085             
 last_mod                           | 2019-04-07 11:20:00  
 generif_text                       | Ne

In [14]:
df_con_conteo = resultado.withColumn("conteo_de_publicaciones", size(split(col("pubmed_id_list"), ",")))
conteo_valores = df_con_conteo.groupBy("conteo_de_publicaciones").agg(count("*").alias("conteo_de_valores"))
conteo_valores.sort(col('conteo_de_valores').desc()).show()

+-----------------------+-----------------+
|conteo_de_publicaciones|conteo_de_valores|
+-----------------------+-----------------+
|                      1|           885579|
|                      2|           191501|
|                      3|            90871|
|                      4|            36523|
|                      5|            22110|
|                      6|            15352|
|                      7|            11231|
|                      8|             9529|
|                      9|             6513|
|                     10|             5348|
|                     11|             3903|
|                     12|             3635|
|                     13|             2760|
|                     14|             2105|
|                     15|             1822|
|                     16|             1483|
|                     17|             1309|
|                     18|             1131|
|                     19|              981|
|                     23|       

In [15]:
df_2 = df_con_conteo.where(~(col('conteo_de_publicaciones') == 1))

In [16]:
df_2.count()

423261

In [17]:
df_filtrado = df_2.drop_duplicates(['gene_id','interactant_id','Entrez Gene Interactor A','Entrez Gene Interactor B'])

In [18]:
df_filtrado.count()

83592

In [19]:
df_filtrado.show(3, vertical = True)

-RECORD 0--------------------------------------------------
 #tax_id                            | 9606                 
 gene_id                            | 2                    
 accn.vers2                         | -                    
 name3                              | -                    
 keyphrase                          | -                    
 tax_id                             | 9606                 
 interactant_id                     | 56983                
 interactant_id_type                | GeneID               
 accn.vers8                         | -                    
 name9                              | -                    
 complex_id                         | -                    
 complex_id_type                    | -                    
 complex_name                       | -                    
 pubmed_id_list                     | 28514442,33961781    
 last_mod                           | 2021-11-07 10:23:00  
 generif_text                       | Af

In [20]:
df_homo_sapiens.where( (col('Entrez Gene Interactor A') == '2' ) & (col('Entrez Gene Interactor B') == '56983' ) ).show(vertical = True)

-RECORD 0--------------------------------------------------
 #BioGRID Interaction ID            | 2260538              
 Entrez Gene Interactor A           | 2                    
 Entrez Gene Interactor B           | 56983                
 BioGRID ID Interactor A            | 106524               
 BioGRID ID Interactor B            | 121300               
 Systematic Name Interactor A       | -                    
 Systematic Name Interactor B       | MDS010               
 Official Symbol Interactor A       | A2M                  
 Official Symbol Interactor B       | POGLUT1              
 Synonyms Interactor A              | A2MD|CPAMD5|FWP00... 
 Synonyms Interactor B              | C3orf9|CLP46|KDEL... 
 Experimental System                | Affinity Capture-MS  
 Experimental System Type           | physical             
 Author                             | Huttlin EL (2017)    
 Publication Source                 | PUBMED:28514442      
 Organism ID Interactor A           | 96

In [21]:
# En la de Biogrid hay un registro para cada publicación, mientras que en la de interactions las publicaciones están en una lista.

In [45]:
import re
from Bio import Entrez
import time

def get_gene_sequence(gene_id):
    Entrez.email =  "rdelanuezmo@alumni.unav.es"
    try:
        handle = Entrez.efetch(db="gene", id=gene_id, rettype="fasta", retmode="text")
        sequence_data = handle.read()
        handle.close()
        return sequence_data
    except Exception as e:
        print(f'Error en {gene_id}:{e}')
        return None

def extract_annotation(sequence_data):
    # Define a regular expression pattern to match the annotation line
    pattern = r'Annotation:\s+(.*?)\s*\('
    
    # Use re.search to find the annotation in the sequence data
    match = re.search(pattern, sequence_data)
    
    if match:
        annotation = match.group(1)
        return annotation
    else:
        return None

if __name__ == '__main__':
    gene_id = "2"  # Replace with your gene ID, e.g., 7157 for TP53
    sequence = get_gene_sequence(gene_id)
    annotation = extract_annotation(sequence)
    
    
    if annotation:
        print("Annotation:", annotation)
    else:
        print("Annotation not found in the sequence data.")


Annotation: Chromosome 12 NC_000012.12


In [23]:
df_int.where( (col('#tax_id') == 9606) & (col('tax_id') == 9606)).count()

1758660

In [24]:
resultado_2 = df_int.where( (col('#tax_id') == 9606) & (col('tax_id') == 9606)).join(df_homo_sapiens, (df_int.where( (col('#tax_id') == 9606) & (col('tax_id') == 9606))["gene_id"] == df_homo_sapiens["Entrez Gene Interactor A"])\
                        & (df_int.where( (col('#tax_id') == 9606) & (col('tax_id') == 9606))["interactant_id"] == df_homo_sapiens["Entrez Gene Interactor B"]), "leftanti")

In [25]:
resultado_2.show(vertical = True)

-RECORD 0-----------------------------------
 #tax_id             | 9606                 
 gene_id             | 1                    
 accn.vers2          | -                    
 name3               | -                    
 keyphrase           | -                    
 tax_id              | 9606                 
 interactant_id      | 3320                 
 interactant_id_type | GeneID               
 accn.vers8          | -                    
 name9               | -                    
 complex_id          | -                    
 complex_id_type     | -                    
 complex_name        | -                    
 pubmed_id_list      | 31273033             
 last_mod            | 2021-05-09 12:10:00  
 generif_text        | Reconstituted Com... 
 interaction_id      | 109552               
 interaction_id_type | BioGRID              
-RECORD 1-----------------------------------
 #tax_id             | 9606                 
 gene_id             | 1                    
 accn.vers

In [26]:
df_homo_sapiens.where( (col('Entrez Gene Interactor A') == 1) & (col('Entrez Gene Interactor B') == 3320)).show(1, vertical = True)

(0 rows)



In [27]:
# Viendo esto yo me quedaría con la primera base de datos

df_homo_int = df_int.where( (col('#tax_id') == 9606) & (col('tax_id') == 9606))

In [28]:
df_con_conteo = df_homo_int.withColumn("conteo_de_publicaciones", size(split(col("pubmed_id_list"), ",")))
conteo_valores = df_con_conteo.groupBy("conteo_de_publicaciones").agg(count("*").alias("conteo_de_valores"))
conteo_valores.sort(col('conteo_de_valores').desc()).show()

+-----------------------+-----------------+
|conteo_de_publicaciones|conteo_de_valores|
+-----------------------+-----------------+
|                      1|          1517150|
|                      2|           157403|
|                      3|            50866|
|                      4|            13676|
|                      5|             6679|
|                      6|             3783|
|                      7|             2480|
|                      8|             1718|
|                      9|             1135|
|                     10|              796|
|                     11|              582|
|                     12|              489|
|                     13|              333|
|                     14|              235|
|                     15|              207|
|                     16|              149|
|                     17|              114|
|                     18|              101|
|                     19|               83|
|                     21|       

In [29]:
df_2 = df_con_conteo.where(~(col('conteo_de_publicaciones') == 1))

In [30]:
df_2.count()

# Número de interacciones que aparecen en más de una publicación 

241510

In [31]:
conteo_valores_id = df_2.groupBy("gene_id", "interactant_id").agg(count("*").alias("conteo_de_valores"))
conteo_valores_id.sort(col('conteo_de_valores').desc()).show()

+-------+--------------+-----------------+
|gene_id|interactant_id|conteo_de_valores|
+-------+--------------+-----------------+
|   1457|          1460|                8|
|   4909|          4915|                6|
|   6607|          8487|                6|
|   1080|          9368|                5|
|   2353|          3725|                5|
|   1956|          6464|                5|
|   4914|          6464|                5|
|   2885|          6464|                5|
|    154|          9368|                5|
|   3065|         25942|                5|
|   2885|          5781|                5|
|    207|           367|                5|
|   1956|          5921|                5|
|   1869|          5925|                5|
|   1956|          4582|                5|
|   5144|         10399|                5|
|    545|           672|                5|
|   4582|          6714|                5|
|   1956|          5335|                5|
|   5159|          5921|                5|
+-------+--

In [32]:
df_2.where( (col('gene_id') == 1457) & (col('interactant_id') == 1460) ).show(vertical = True)

-RECORD 0---------------------------------------
 #tax_id                 | 9606                 
 gene_id                 | 1457                 
 accn.vers2              | -                    
 name3                   | -                    
 keyphrase               | -                    
 tax_id                  | 9606                 
 interactant_id          | 1460                 
 interactant_id_type     | GeneID               
 accn.vers8              | -                    
 name9                   | -                    
 complex_id              | -                    
 complex_id_type         | -                    
 complex_name            | -                    
 pubmed_id_list          | 9188720,9571630,1... 
 last_mod                | 2023-07-09 12:12:00  
 generif_text            | Affinity Capture-... 
 interaction_id          | 107843               
 interaction_id_type     | BioGRID              
 conteo_de_publicaciones | 35                   
-RECORD 1-----------

In [33]:
# Juntamos los identificadores de los genes en un dataframe
from pyspark.sql.types import IntegerType

a_1 = df_2.select('gene_id')
a_2 = df_2.select('interactant_id')
a_1 = a_1.withColumnRenamed("gene_id", "id")
a_2 = a_2.withColumnRenamed("interactant_id", "id")
a = a_1.union(a_2).drop_duplicates(['id'])
a = a.withColumn("id", a["id"].cast(IntegerType()))

In [34]:
a.show()

+----+
|  id|
+----+
| 467|
| 675|
| 829|
|1159|
| 125|
| 800|
| 944|
| 666|
| 919|
|  51|
| 307|
| 475|
| 613|
| 718|
| 205|
| 334|
| 581|
| 740|
| 747|
| 462|
+----+
only showing top 20 rows



In [35]:
a.orderBy(a['id'].desc()).show()

+---------+
|       id|
+---------+
|105375355|
|102724652|
|102288414|
|102157402|
|101060200|
|100652748|
|100532731|
|100529063|
|100528064|
|100528030|
|100526740|
|100526737|
|100526693|
|100507436|
|100507290|
|100506742|
|100506658|
|100505741|
|100423062|
|100302090|
+---------+
only showing top 20 rows



In [36]:
# Número de identificadores de genes únicos
a.count()

13791

In [37]:
a_final = a.toPandas()

In [38]:
a_final

,id
0,467
1,675
2,829
3,1159
4,125
...,...
13786,115761
13787,57017
13788,283254
13789,375567


In [39]:
a_final.describe()

,id
count,1.379100e+04
mean,3.963158e+05
std,5.796083e+06
min,1.000000e+00
25%,6.047500e+03
50%,2.330900e+04
75%,7.972350e+04
max,1.053754e+08


In [ ]:
a_final['secuencia'] = a_final['id'].map(get_gene_sequence)

Error en 154:HTTP Error 400: Bad Request
Error en 29:HTTP Error 400: Bad Request
Error en 1021:HTTP Error 400: Bad Request
Error en 629:HTTP Error 400: Bad Request
Error en 408:HTTP Error 400: Bad Request
Error en 1106:HTTP Error 400: Bad Request
Error en 185:IncompleteRead(386 bytes read)
Error en 335:HTTP Error 400: Bad Request
Error en 41:HTTP Error 400: Bad Request
Error en 941:HTTP Error 400: Bad Request
Error en 575:IncompleteRead(348 bytes read)
Error en 2948:HTTP Error 400: Bad Request
Error en 3107:HTTP Error 400: Bad Request
Error en 2773:HTTP Error 400: Bad Request
Error en 1288:IncompleteRead(454 bytes read)
Error en 1968:HTTP Error 400: Bad Request
Error en 2566:HTTP Error 400: Bad Request
Error en 1523:IncompleteRead(589 bytes read)
Error en 2824:HTTP Error 400: Bad Request
Error en 1413:HTTP Error 400: Bad Request
Error en 2962:HTTP Error 400: Bad Request
Error en 3949:HTTP Error 400: Bad Request
Error en 5079:HTTP Error 400: Bad Request
Error en 4580:HTTP Error 400: Bad